In [26]:
import copy
steps = []
def extend_tree(G, k, T, I, B, L, X, recall):
    #record all steps
    recall = 0
    
    # Halt - yes
    if len(L) + len(B) >= k:
        return True
    
    # Halt - No
    if len(B) == 0:
        steps[-1][5] += 1
        return False
    
    # Simplication rule：If exist v∈B with N_G(v) ∩ X = 0, then move v to L
    B_to_L = {v for v in B if not set(G.neighbors(v)) & X}
    for v in B_to_L:
        B.remove(v)
        L.add(v)
        steps.append([copy.deepcopy(T), copy.deepcopy(I), copy.deepcopy(B), copy.deepcopy(L), copy.deepcopy(X), copy.deepcopy(recall), 2])

    #Select u from B
    for u in list(B):
        
        # branch 1: try to move a vertex from B to L
        B.remove(u)
        L.add(u)  
        steps.append([copy.deepcopy(T), copy.deepcopy(I), copy.deepcopy(B), copy.deepcopy(L), copy.deepcopy(X), copy.deepcopy(recall), 3])
        if extend_tree(G, k, T, I, B, L, X, recall):
            return True
        # recall branch 1
        L.remove(u)
        B.add(u)
        steps.append([copy.deepcopy(T), copy.deepcopy(I), copy.deepcopy(B), copy.deepcopy(L), copy.deepcopy(X), copy.deepcopy(recall), 4])

        # branch 2: handle u as a internal vertex (Branching Lemma)
        original_u = u  # save the original vertex
        added_edges = []  # save added edges
        B.remove(u)
        I.add(u)
        neighbors_X = set(G.neighbors(u)) & X
        
        #Follow Path Lemma
        while len(neighbors_X) == 1:
            v = neighbors_X.pop()
            X.remove(v)
            T.add_edge(u, v)
            added_edges.append((u, v))
            neighbors_X = set(G.neighbors(v)) & X
            u = v  # update v as new u(current vertex)
            if len(neighbors_X) == 1:
                I.add(u)
            elif len(neighbors_X) != 1:
                B.add(u)
            steps.append([copy.deepcopy(T), copy.deepcopy(I), copy.deepcopy(B), copy.deepcopy(L), copy.deepcopy(X), copy.deepcopy(recall), 5])
        
        B.discard(u)
        # process branch 2
        for v in neighbors_X:
            T.add_edge(u, v)
            added_edges.append((u, v))
            B.add(v)
            X.remove(v)
            steps.append([copy.deepcopy(T), copy.deepcopy(I), copy.deepcopy(B), copy.deepcopy(L), copy.deepcopy(X), copy.deepcopy(recall), 6])
    
        if extend_tree(G, k, T, I, B, L, X, recall):
            return True
    
        # recall branch 2
        for u, v in added_edges:
            T.delete_edge((u, v))  # delete added edges
            I.discard(v)
            X.add(v)
        I.discard(original_u)  # remove original u
        B.add(original_u)  # add original u to B
        X.update(neighbors_X)  # update unused vertices to X
        for v in neighbors_X:
            B.discard(v)  # remove vertices from B
        steps.append([copy.deepcopy(T), copy.deepcopy(I), copy.deepcopy(B), copy.deepcopy(L), copy.deepcopy(X), copy.deepcopy(recall), 7])
    steps[-1][5] += 1
    return False

def max_leaf_spanning_tree(G, k):
    T = Graph()
    I = set()
    L = set()
    B = set()
    X = set(G.vertices())
    recall = 0
    # save initial state
    for r in G.vertices():
        steps.append([Graph(), set(), set(), set(), set(G.vertices()), 0, 8])
        T = Graph() #a tree in G
        T.add_vertex(r) #add initial vertex to T
        I = set()  # the internal vertices of T, with r∈I
        L = set()  # the remaining leaves of T
        B = {r}  # a subset of the leaves of T where T may be extended: the boundary set
        X = set(G.vertices()) - {r}  # the external vertices V\V(T)
        steps.append([copy.deepcopy(T), copy.deepcopy(I), copy.deepcopy(B), copy.deepcopy(L), copy.deepcopy(X), copy.deepcopy(recall), 1])
        if extend_tree(G, k, T, I, B, L, X, recall):
            return True, T
    return False, None


In [27]:
# use example
G = Graph([('a', 'b'), ('b', 'c'), ('c', 'a'), ('c', 'd'), ('d', 'e')])  
k = 3  # target least number of leaves
result, T = max_leaf_spanning_tree(G, k)
if result:
    print("Exist a spanning tree with maximum leaf number of {}.".format(k))
else:
    print("Not Exist a spanning tree with maximum leaf number of {}.".format(k))

Exist a spanning tree with maximum leaf number of 3.


In [32]:
@interact
def show_step(step=slider(0, len(steps)-1, 1, label="Step")):
    T, I, B, L, X, recall, operation_type = steps[step]
    # vertex colors
    vertex_colors = {'green': list(L), 'red': list(B), 'white': list(X), 'yellow': list(I)}

    # edge_colors
    edge_colors = {'yellow': T.edges(labels=False)}
    
    # plot graph
    G_plot = G.plot(vertex_colors=vertex_colors, edge_colors=edge_colors, layout='circular')
    
    # show current steps of algorithm
    
    print("The current tree T are represented in yellow, the nodes in B are in red, the nodes in L are in green,")
    print(" and the remaining unchanged parts(X) set to white.\n")
    print(f"Current state: I:{I if I else ' Null'}, B:{B if B else ' Null'}, L:{L if L else ' Null'}, X:{X if X else ' Null'}, k:{k}\n")
    
    # operation explanation
    if operation_type == 1:
        print(f"This state we choose a new vertex as a start vertex.\n")
    elif operation_type == 2:
        print(f"Simplication rule：If exist v∈B with N_G(v) ∩ X = 0, then move v to L.")
        print(f"This means all vertices in B that can not be expanded will be moved to L.\n")
    elif operation_type == 3:
        print(f"This step we go to branch 1: try to move a vertex from B to L.\n")
    elif operation_type == 4:
        print(f"Recall the branch 1 operation: move the vertex back to B.\n")
    elif operation_type == 5:
        print(f"This step will process follow path lemma.\n")
    elif operation_type == 6:
        print(f"This step we go to branch 2: handle current vertex u as a internal vertex (Branching Lemma).")
        print(f"Then, put all vertices adjacent to the current vertex u and not yet processed into set B.\n")
    elif operation_type == 7:
        print(f"Recall the branch 2 operation: Branching Lemma.\n")
    elif operation_type == 8:
        print(f"This step is initial state of the graph.\n")
    
    # recall info
    if recall == 1:
        print(f"The algorithm has hit a dead end, this state will recall {recall} steps.\n")
    elif recall == 3:
        print(f"This initial vertex is not available, the state will recall to initial graph and choose a new initial vertex.\n")
    
    #result
    if step == len(steps)-1:
        if result:
            print(f"The number of sum of vertices in B and vertices in L has reached k:{k}.")
            print(f"YES. The branching algorithm find a maximum leaf spanning tree with at least {k} leaves.\n")
        else:
            print(f"The branching algorithm has test all viable tree, but can't reach k:{k}.")
            print(f"NO. The branching algorithm find out there is no maximum leaf spanning tree in G.\n")
    show(G_plot)

Widget Javascript not detected.  It may not be installed or enabled properly. Reconnecting the current kernel may help.
